In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=5
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i_fnsd.cmd' % k)
solverF.init_grid()
solverF.init_flux_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFNSD%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, transient=True)
solverS.output_file = 'solid_outputs/resFNSD%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_temperature_interface(tag=interface_tag, init_values=T_bot)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=1e-6)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FNSD%i.log' % k, mode='w')
flog.write('Fluid Neumann with solid Dirichlet setting, study case: %i\n' % k)
flog.close()
flog = open('FNSD%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # backup previous interface values
        tempS.backup()
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface flux
        fluxS.curr[:] = solverS.get_interface_flux(tag=interface_tag)
        
        # mapping
        fluxF.curr[:] = fluxS.curr[s2f]
        # update fluid flux interface
        solverF.put_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis,
            data=fluxF.curr
        )
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface temperature
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        tempS.curr[:] = tempF.curr[f2s]
        
        # update residual
        tempS.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempS)
        if is_conv or pc_counts >= max_pc_steps:
            break
        else:
            # if not converge, then underrelaxation and update to solid then restore
            under_relax.determine_omega(tempS)
            under_relax.update_solution(tempS)
            solverS.update_temperature_interface(interface_tag, tempS.curr)
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=11.
step=1, pc_iterations=8.
step=2, pc_iterations=8.
step=3, pc_iterations=8.
step=4, pc_iterations=8.
step=5, pc_iterations=8.
step=6, pc_iterations=8.
step=7, pc_iterations=8.
step=8, pc_iterations=8.
step=9, pc_iterations=8.
step=10, pc_iterations=8.
step=11, pc_iterations=8.
step=12, pc_iterations=7.
step=13, pc_iterations=7.
step=14, pc_iterations=7.
step=15, pc_iterations=7.
step=16, pc_iterations=7.
step=17, pc_iterations=7.
step=18, pc_iterations=7.
step=19, pc_iterations=7.
step=20, pc_iterations=7.
step=21, pc_iterations=7.
step=22, pc_iterations=7.
step=23, pc_iterations=7.
step=24, pc_iterations=7.
step=25, pc_iterations=7.
step=26, pc_iterations=7.
step=27, pc_iterations=7.
step=28, pc_iterations=7.
step=29, pc_iterations=7.
step=30, pc_iterations=7.
step=31, pc_iterations=7.
step=32, pc_iterations=7.
step=33, pc_iterations=7.
step=34, pc_iterations=7.
step=35, pc_iterations=7.
step=36, pc_iterations=7.
step=37, pc_iterations=7.
step=38, pc_iteration

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFNSD%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.66742027 0.68653083 0.70147072 0.71061406 0.71827772 0.7251652
 0.73120477 0.7366012  0.74168771 0.74617214 0.75030266 0.75408655
 0.75767832 0.7609719  0.76405022 0.7668878  0.76958304 0.77209031
 0.77440964 0.77661539 0.77866069 0.78058544 0.78239503 0.78407632
 0.78566771 0.78713995 0.78854301 0.78982156 0.79099593 0.79213978
 0.79320574 0.79418655 0.79510199 0.79595301 0.79675214 0.79750121
 0.79820501 0.7988304  0.79942546 0.79996434 0.80046524 0.80092388
 0.80134514 0.80172867 0.80207285 0.8023808  0.80267693 0.80297079
 0.80324562 0.80348919 0.80359833 0.80367068 0.80377728 0.80388404
 0.80399716 0.80403743 0.80403868 0.80402632 0.80399828 0.80395282
 0.8038774  0.80378137 0.80362846 0.80344837 0.80325139 0.80303277
 0.80278112 0.80249407 0.80216124 0.80178047 0.8014086  0.8009789
 0.80048602 0.79996264 0.79937093 0.79873086 0.7980295  0.79727257
 0.79643935 0.79553744 0.79454189 0.79346021 0.79228911 0.7910194
 0.78962057 0.78806924 0.78641868 0.78457228 0.782517   0.7802815